In [ ]:
pip install autokeras

     |████████████████████████████████| 174kB 23.7MB/s 
     |████████████████████████████████| 71kB 12.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=2f90b2ab84be72a51551e27f140ee0ff75b6a48855cf42714547f2877b9f09ee
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=f6f70b7457350b313cbd123258c5c5d5461119192d6a5a912309e217e26c05a9
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
import autokeras as ak

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[-1], 'GPU')

## A Simple Example

### Load MNIST dataset

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Training image shape:', x_train.shape) # (60000, 28, 28)
print('Training label shape:', y_train.shape) # (60000,)
print('First five training labels:', y_train[:5]) # array([5 0 4 1 9], dtype=uint8)

11493376/11490434 [==============================] - 0s 0us/step
Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


### Run the ImageClassifier

In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(max_trials=2) # It tries two different models.

# Feed the image classifier with training data 
# 20% of the data is used as validation data by default for tuning
# the process may run for a bit long time, please try to use GPU
clf.fit(x_train, y_train, epochs=3) # each model is trained for three epochs

Trial 2 Complete [00h 26m 02s]
val_loss: 0.4253802001476288

Best val_loss So Far: 0.04771251976490021
Total elapsed time: 00h 26m 32s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2961 - accuracy: 0.9101
Epoch 2/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0790 - accuracy: 0.9763
Epoch 3/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0596 - accuracy: 0.9815
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


### Get the summarized results during the tuning process (return the best 10 models if existed)

In [ ]:
clf.tuner.results_summary()

Results summary
Results in ./image_classifier
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
image_block_1/block_type: vanilla
image_block_1/normalize: True
image_block_1/augment: False
image_block_1/conv_block_1/kernel_size: 3
image_block_1/conv_block_1/num_blocks: 1
image_block_1/conv_block_1/num_layers: 2
image_block_1/conv_block_1/max_pooling: True
image_block_1/conv_block_1/separable: False
image_block_1/conv_block_1/dropout: 0.25
image_block_1/conv_block_1/filters_0_0: 32
image_block_1/conv_block_1/filters_0_1: 64
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0.5
optimizer: adam
learning_rate: 0.001
Score: 0.04718967527151108
Trial summary
Hyperparameters:
image_block_1/block_type: resnet
image_block_1/normalize: True
image_block_1/augment: True
image_block_1/image_augmentation_1/horizontal_flip: True
image_block_1/image_augmentation_1/vertical_flip: True
image_block_1/image_augm

### Retrieve best model

In [ ]:
best_model = clf.export_model()
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 28, 28)            0         
_________________________________________________________________
expand_last_dim (ExpandLastD (None, 28, 28, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)       


### Predict with the best model.


In [ ]:
predicted_y = clf.predict(x_test)
print(predicted_y)

[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]


In [ ]:
type(predicted_y[0][0])

numpy.str_

### Evaluate the best model on the test data.

In [ ]:

test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', test_acc)


Test accuracy:  0.988099992275238


### Save and load model

In [ ]:
best_model.save("model_autokeras")

INFO:tensorflow:Assets written to: model_autokeras/assets


In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras") # , custom_objects=ak.CUSTOM_OBJECTS

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', test_acc)

[[1.7567700e-10 3.5295128e-11 1.2154676e-08 ... 9.9999976e-01
  2.6258207e-10 2.8395709e-08]
 [6.6249015e-08 6.2428267e-06 9.9999332e-01 ... 1.6883059e-13
  1.5260402e-09 1.2889657e-12]
 [4.9405344e-08 9.9961913e-01 1.8153445e-05 ... 3.0194589e-05
  2.4773601e-06 5.2960836e-08]
 ...
 [4.9813994e-13 4.9129406e-10 1.7102680e-12 ... 5.6733228e-07
  1.4088409e-06 1.0516960e-06]
 [6.1794730e-10 2.8649080e-12 1.8733258e-10 ... 7.1567530e-11
  1.2792475e-05 2.6690681e-09]
 [4.0928891e-09 8.4695960e-13 4.1557588e-08 ... 4.6263005e-14
  1.5417026e-08 1.1706852e-10]]
Test accuracy:  0.988099992275238


## Validation Data

By default, AutoKeras use the last 20% of training data as validation data. As shown in the example below, you can use validation_split to specify the percentage.

In [ ]:
clf.fit(x_train,
        y_train,
        # Split the training data and use the last 15% as validation data.
        validation_split=0.15,epochs=3)

You can also use your own validation set instead of splitting it from the training data with validation_data.

In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(x_train,
        y_train,
        # Use your own validation set.
        validation_data=(x_val, y_val),epochs=3)

## Data Format

The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel dimension. The images in the MNIST dataset do not have the channel dimension. Each image is a matrix with shape (28, 28). AutoKeras also accepts images of three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28, 28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e. strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and 1s.

So if you prepare your data in the following way, the ImageClassifier should still work.

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
# x_train = x_train.reshape(x_train.shape + (1,))
# x_test = x_test.reshape(x_test.shape + (1,))

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


# One-hot encode the labels.
import numpy as np
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape) # (60000, 28, 28, 1)
print(y_train.shape) # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

(60000, 28, 28, 1)
(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


We also support using tf.data.Dataset format for the training data. In this case, the images would have to be 3-dimentional. The labels have to be one-hot encoded for multi-class classification to be wrapped into tensorflow Dataset.

In [ ]:
import tensorflow as tf
train_set = tf.data.Dataset.from_tensor_slices(((x_train, ), (y_train, )))
test_set = tf.data.Dataset.from_tensor_slices(((x_test, ), (y_test, )))

clf = ak.ImageClassifier(max_trials=2)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

### Configurate search process

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

clf = ak.ImageClassifier(max_trials=2, 
                         loss='categorical_crossentropy', 
                         metrics=['accuracy'],
                         objective='val_accuracy',
                        )

clf.fit(x_train, y_train, 
        validation_split=0.15,
        epochs=3, verbose=2, )

Trial 2 Complete [00h 26m 19s]
val_accuracy: 0.9221529960632324

Best val_accuracy So Far: 0.9883229732513428
Total elapsed time: 00h 26m 46s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
1875/1875 - 5s - loss: 0.1549 - accuracy: 0.9531
Epoch 2/3
1875/1875 - 5s - loss: 0.0724 - accuracy: 0.9782
Epoch 3/3
1875/1875 - 5s - loss: 0.0605 - accuracy: 0.9815
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


### Customize search objective and metrics

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


import kerastuner
def my_metric(y_true, y_pred):
    correct_labels = tf.cast(y_true == y_pred, tf.float32)
    return tf.reduce_mean(correct_labels, axis=-1) 

clf = ak.ImageClassifier(
    seed=42,
    max_trials=2,
    loss='categorical_crossentropy',
    # Wrap the function into a Keras Tuner Objective 
    # and pass it to AutoKeras.

    # Direction can be 'min' or 'max'
    # meaning we want to minimize or maximize the metric.

    # 'val_my_metric' is just add a 'val_' prefix
    # to the function name or the metric name.
    objective=kerastuner.Objective('val_my_metric', direction='max'),
    # Include it as one of the metrics.
    metrics=[my_metric],
)

clf.fit(x_train, y_train, 
        validation_split=0.15,
        epochs=3)

Trial 1 Complete [00h 27m 54s]
val_my_metric: 0.0009341632248833776

Best val_my_metric So Far: 0.0009341632248833776
Total elapsed time: 00h 27m 54s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
Tensor("Cast_1:0", shape=(None, 10), dtype=float32)
Tensor("Cast_1:0", shape=(None, 10), dtype=float32)
1875/1875 [==============================] - 624s 330ms/step - loss: 0.7643 - my_metric: 2.8465e-05
Epoch 2/3
1875/1875 [==============================] - 619s 330ms/step - loss: 0.2722 - my_metric: 4.1554e-04
Epoch 3/3
1875/1875 [==============================] - 618s 329ms/step - loss: 0.2178 - my_metric: 9.2383e-04
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/preprocessor/#normalization-class),
[ImageAugmentation](/preprocessor/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/head/#classificationhead-class).